In [6]:
import torch
import torch.nn as nn
import numpy as np


- softmax with numpy

In [17]:
import torch
import torch.nn as nn
import numpy as np

def softmax(x):
    return np.exp(x) / np.sum(np.exp(x), axis=0)
# 数组 / 标量，用到了向量化操作，可以不用python的循环

x = np.array([2.0, 1.0, 0.1])
outputs_x = softmax(x)
print(f'softmax with numpy:{outputs_x}')

softmax with numpy:[0.65900114 0.24243297 0.09856589]


- softmax with pytorch

In [23]:
import torch
import torch.nn as nn
import numpy as np

y = torch.tensor([2.0, 1.0, 0.1], dtype=torch.float32)
outputs_y_1 = torch.softmax(y, dim=0)    # 要指定轴
outputs_y_2 = y.softmax(dim=0)           # 另一种方法
print(f'softmax_pytorch: \n{outputs_y_1} \n{outputs_y_2}')



softmax_pytorch: 
tensor([0.6590, 0.2424, 0.0986]) 
tensor([0.6590, 0.2424, 0.0986])


- cross-entropy
